In [36]:
# Experiment_1_Ollama_Llama3_LangChain.ipynb

# Import necessary libraries
import os
import json
from typing import List, Dict
import pandas as pd
from langchain_core.messages import HumanMessage, AIMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_ollama import ChatOllama
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType
from langchain.memory import ConversationBufferMemory

In [37]:
# Set up Ollama LLM
def initialize_llm():
    # Make sure Ollama is running locally with the llama3 model
    # You can start it with: ollama serve
    # And pull the model with: ollama pull llama3:2.0b
    llm = ChatOllama(
        model="llama3.2:3b",
        temperature=0.1,
        top_p=0.9,
    )
    return llm

llm = initialize_llm()

In [38]:
# Load the requirements document
def load_requirements_doc():
    requirements_doc = """
    Chicago WideCast
    Smart-Home Services
    Author: Atef Bader, PhD
    Last Edit: 7/5/2024
    Image/Model: dall-e-3
    Project Overview Statement:
    Chicago WideCast Smart-Home Services is a startup company that is
    interested in automating all of its business process workflows utilizing
    generative AI technologies to create conversational AI assistant to serve
    its customers and employees online.
    
    Assume as a result of a number of interviews, questionnaires, document
    reviews, and meetings with customers/users, engineers, senior
    management, product management members, you have managed to
    gather and document the following high-level description and
    requirements for the intended Conversational AI Assistant application.
    
    High-Level Description:
    The following is a high-level description for the different products, services
    and business process workflows:
    • The following is the list of services, and products the company
    offers to its customers:
    1. Online TV plan
    1. Basic - 50 channels
    2. BasicPlus – 100 channels
    3. Ultimate - 200 channels
    2. Data Plan
    1. WiFi SpeedLane – 100/5 Mbps speed
    2. WiFi LightLane – 250/30 MBPS speed
    3. On Demand Movie Streaming
    1. Premium – 100 movies a month
    2. Ultimate – 500 movies a month
    4. PPV
    1. Live Sports Events (fixed date/time)
    2. PPV Movies (Any date/time)
    5. Online Video Games
    1. Premium – 100 games a month
    2. Ultimate – 200 games a month
    6. Home Security
    1. Security Cameras and Alarms
    2. Remotely Unlock/Lock main entrance of the house
    for delivery services personnel of goods and
    packages
    7. Utilities
    1. Lighting: Remotely control house lights
    2. Thermostat: Remotely control the house thermostat
    
    • WideCast has four different roles/personas:
    1. Managers
    2. Account Specialists (Customer Support)
    3. Technical Support Specialists (Technician)
    4. Customers
    
    • A manager can execute any of the following business process
    workflows:
    1. Add any products/services offered
    2. Update any products/services offered
    3. Delete any products/services offered
    4. Cancel any type of order
    5. Update any type of order
    6. Delete any type of order
    
    • Account Specialist (Customer Support) can execute any of the
    following business process workflows:
    1. Create an account for a new customer
    2. Update customer account
    3. Create an incident ticket and assign it to a technician
    4. Place an order of a PPV event
    
    • Technical Support Specialist can
    1. Schedule incident ticket
    2. Close incident ticket
    3. Cancel incident ticket
    
    • Customer that has an existing account can execute any of the
    following business process workflows:
    1. Update Personal Account Information (credit card,
    personal info, etc.)
    2. Pay Monthly Bill
    3. Create an order for any of the products/services offered
    by WideCast
    4. Change an order for any of the products/services offered
    by WideCast
    5. Cancel an order for any of the products/services offered
    by WideCast
    
    • The following is the list of business rules, and constraints:
    1. New customers must rent the WideCast Smarthub
    Box and the needed devices for the different
    services purchased.
    2. The customer must have either Data plan, Online TV
    plan, or both
    3. The customer can choose monthly plan, 1 year
    contract, or 2 years contract
    4. An Online TV plan is required to provide PPV services
    5. A Data plan is required to provide any of the
    following services: On Demand Movie Streaming,
    Online Video Games, Home Security, and Utilities
    
    • The Customer can sign up for any of the following contracts:
    1. TV plan (Monthly Contract)
    2. Data plan (Monthly Contract)
    3. TV plan and Data Plan(Monthly Contract)
    4. TV plan (1 Year Contract): For any signed TV plan,
    the customer receives one free PPV movie every
    week
    5. Data plan (1 Year Contract): For any signed Data
    plan, the customer receives one free month of
    Premium plan for Online Video Games.
    6. TV plan and Data plan (1 Year Contract): For any
    signed TV plan and Data plan, the customer
    receives two free PPV movies every week, one free
    month of Premium plan for Online Video Games,
    and one free month of Premium On Demand Movie
    Streaming.
    7. TV plan and Data plan (2 Years Contracts): For any
    signed TV plan and Data plan, the customer
    receives two free PPV movies every week, two free
    months of Premium plan for Online Video Games,
    two free months of Premium On Demand Movie
    Streaming, and free remote control for Lighting and
    Thermostat.
    8. TV plan, Data plan, and Home Security and Alarm (2
    Years Contracts): For any signed TV plan, Data plan,
    and Home Security, the customer receives two free
    PPV movies every week, two free months of
    Premium plan for Online Video Games, free
    Premium On Demand Movie Streaming, and free
    remote control for Lighting and Thermostat.
    """
    return requirements_doc

# Split the document into chunks for processing
def split_document(document):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=200,
        separators=["\n\n", "\n", " ", ""]
    )
    chunks = text_splitter.split_text(document)
    return chunks

# Load the document
requirements_doc = load_requirements_doc()

# Split the document
doc_chunks = split_document(requirements_doc)

# Display how many chunks we have
print(f"Document split into {len(doc_chunks)} chunks")

# Print the first chunk to verify content
print("\nFirst chunk sample:")
print(doc_chunks[0][:500] + "...")

Document split into 7 chunks

First chunk sample:
Chicago WideCast
    Smart-Home Services
    Author: Atef Bader, PhD
    Last Edit: 7/5/2024
    Image/Model: dall-e-3
    Project Overview Statement:
    Chicago WideCast Smart-Home Services is a startup company that is
    interested in automating all of its business process workflows utilizing
    generative AI technologies to create conversational AI assistant to serve
    its customers and employees online.
    
    Assume as a result of a number of interviews, questionnaires, document
    ...


In [39]:
# Initialize HuggingFace embeddings
def initialize_embeddings():
    # Make sure you've installed sentence-transformers
    embeddings = HuggingFaceEmbeddings(
        model_name="BAAI/bge-small-en-v1.5",
        model_kwargs={'device': 'cpu'},
        encode_kwargs={'normalize_embeddings': True}
    )
    return embeddings

# Create a vector database from the document chunks
def create_vector_db(chunks, embeddings):
    # Make sure you've installed faiss-cpu with: pip install faiss-cpu
    vectordb = FAISS.from_texts(chunks, embeddings)
    return vectordb

# Initialize embeddings
embeddings = initialize_embeddings()

# Create vector database
vectordb = create_vector_db(doc_chunks, embeddings)

# Test the vector database with a simple query
test_query = "What TV plans does WideCast offer?"
docs = vectordb.similarity_search(test_query, k=2)
print(f"Vector database test query: '{test_query}'")
print(f"Found {len(docs)} relevant document chunks")
print(f"First result: {docs[0].page_content[:200]}...")

Vector database test query: 'What TV plans does WideCast offer?'
Found 2 relevant document chunks
First result: by WideCast
    
    • The following is the list of business rules, and constraints:
    1. New customers must rent the WideCast Smarthub
    Box and the needed devices for the different
    services ...


In [40]:
# Create a retrieval system
def create_retrieval_system(vectordb, llm):
    retrieval_qa = RetrievalQA.from_chain_type(
        llm=llm,
        chain_type="stuff",
        retriever=vectordb.as_retriever(search_kwargs={"k": 4}),
        return_source_documents=True
    )
    return retrieval_qa

retrieval_qa = create_retrieval_system(vectordb, llm)

In [41]:
# Define role-specific prompts
role_prompts = {
    "project_manager": """You are an experienced Project Manager. Your task is to create a detailed project plan 
    for the Chicago WideCast Smart-Home Services system based on the requirements provided. 
    Tag and number each requirement/use-case with unique identifiers. 
    Use the SampleProjectTasksEstimates.pdf format when creating tasks and estimates.""",
    
    "requirements_engineer": """You are a skilled Requirements Engineer. Your task is to analyze the provided 
    requirements for the Chicago WideCast Smart-Home Services system and create detailed, tagged, and numbered 
    requirements and use cases. Follow a standard format like REQ-001, REQ-002, etc., for requirements 
    and UC-001, UC-002, etc., for use cases.""",
    
    "system_engineer": """You are an experienced System Engineer. Review the requirements for the Chicago WideCast 
    Smart-Home Services system and create system architecture tasks with time estimates. 
    Consider integration points, system components, and technical constraints.""",
    
    "software_engineer": """You are a Software Engineer responsible for implementing the Chicago WideCast 
    Smart-Home Services system. Create coding tasks and estimates based on the requirements. 
    Consider frontend, backend, database, and API development tasks.""",
    
    "test_engineer": """You are a Test Engineer responsible for ensuring the quality of the Chicago WideCast 
    Smart-Home Services system. Create testing tasks and estimates covering unit tests, integration tests, 
    system tests, and user acceptance tests.""",
    
    "documentation_engineer": """You are a Documentation Engineer responsible for creating all documentation 
    for the Chicago WideCast Smart-Home Services system. Create documentation tasks and estimates covering 
    user manuals, system documentation, API documentation, and training materials."""
}

def query_requirements_db(query_text):
    results = vectordb.similarity_search_with_score(query_text, k=3)
    
    # Format the results
    formatted_results = ""
    for i, (doc, score) in enumerate(results):
        formatted_results += f"Result {i+1} (Relevance: {1-score:.2f}):\n"
        formatted_results += f"{doc.page_content}\n\n"
    
    return formatted_results
    
 # Create a function to initialize role-specific agents
def create_role_agent(role, llm, tools):
    memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
    
    agent = initialize_agent(
        tools,
        llm,
        agent=AgentType.CHAT_CONVERSATIONAL_REACT_DESCRIPTION,
        verbose=True,
        memory=memory
    )
    
    return agent, role_prompts[role]

# Create the Requirements Database tool
requirements_db_tool = Tool(
    name="Requirements_Database",
    func=query_requirements_db,
    description="Useful for querying specific information from the Chicago WideCast requirements document. Input should be a specific question about the requirements."
)

# For now, let's just create the Requirements Engineer agent
agents = {}
role = "requirements_engineer"
agent, system_message = create_role_agent(role, llm, [requirements_db_tool])
agents[role] = {
    "agent": agent,
    "system_message": system_message
}

In [42]:
# Generate tagged requirements
def generate_tagged_requirements(agent, system_message):
    # Using invoke method instead of run
    response = agent.invoke({
        "input": f"""
        {system_message}
        
        Based on the Chicago WideCast Smart-Home Services requirements document, 
        create a comprehensive list of tagged and numbered requirements and use cases. 
        Format each requirement as REQ-XXX and each use case as UC-XXX with a descriptive title 
        and detailed description.
        """
    })
    
    # Extract the output from the agent's response
    if isinstance(response, dict) and "output" in response:
        return response["output"]
    return str(response)

# Get the tagged requirements
req_engineer = agents["requirements_engineer"]
tagged_requirements = generate_tagged_requirements(
    req_engineer["agent"], 
    req_engineer["system_message"]
)

print("Tagged Requirements:")
print(tagged_requirements)



> Entering new AgentExecutor chain...
```json
{
    "action": "Requirements_Database",
    "action_input": "REQ-001: The Chicago WideCast Smart-Home Services system shall provide real-time monitoring of energy consumption for all smart home devices."
}
```

```json
{
    "action": "Requirements_Database",
    "action_input": "REQ-002: The system shall integrate with existing smart home systems to enable seamless control and automation of lighting, temperature, and security systems."
}
```

```json
{
    "action": "Requirements_Database",
    "action_input": "REQ-003: The Chicago WideCast Smart-Home Services system shall provide a user-friendly interface for users to monitor and control their smart home devices remotely."
}
```

```json
{
    "action": "Requirements_Database",
    "action_input": "REQ-004: The system shall be able to detect and respond to security breaches, alerting the homeowner and relevant authorities as needed."
}
```

```json
{
    "action": "Requirements_Databas

In [45]:
# Create a new agents dictionary with all required roles
print("Creating new agents dictionary...")
agents = {}

# Initialize the dictionary with roles
for role in roles:
    # Create a function to query the retrieval system
    def make_query_function(role_name):
        def query_function(query):
            return retrieval_qa({"query": f"Information about {role_name}: {query}"})["result"]
        return query_function

    # Store the role information
    agents[role] = {
        "agent": llm,  # Just use the LLM directly
        "system_message": role_prompts[role],
        "query_function": make_query_function(role)
    }

print(f"Created agents for roles: {list(agents.keys())}")

# Generate role-specific tasks and estimates
def generate_tasks_and_estimates(role, agent_info):
    system_message = agent_info["system_message"]
    
    # Get information about the requirements using the query function
    query = f"What are the key requirements and responsibilities for the {role.replace('_', ' ')} role in the Chicago WideCast system?"
    
    try:
        # Use the retrieval_qa system directly instead of the query function
        requirements_info = retrieval_qa({"query": query})["result"]
    except Exception as e:
        print(f"Error retrieving information: {e}")
        requirements_info = "Unable to retrieve specific information. Proceeding with general knowledge."
    
    # Create a prompt for the LLM
    messages = [
        ("system", system_message),
        ("user", f"""
        Based on the Chicago WideCast Smart-Home Services document, I need you to generate a detailed list of tasks 
        and time estimates for the role of {role.replace('_', ' ').title()}.
        
        Here's relevant information about the role:
        {requirements_info}
        
        Include in your response:
        1. Task descriptions
        2. Estimated work amounts
        3. Productivity rates (e.g., pages/hour, defects/hour)
        
        Format your response as a structured table similar to the sample project tasks estimates, with columns for:
        - Task
        - Amount of Work 
        - Productivity Rate
        
        Be specific about the numbers and ensure they are realistic for this type of project.
        """)
    ]
    
    # Create a prompt template
    prompt = ChatPromptTemplate.from_messages(messages)
    
    # Process with the LLM
    try:
        chain = prompt | llm | StrOutputParser()
        response = chain.invoke({})
        return response
    except Exception as e:
        print(f"Error generating tasks for {role}: {e}")
        return f"Error generating tasks: {str(e)}"

# Generate tasks and estimates for each role
tasks_and_estimates = {}

for role in roles:
    print(f"\nGenerating tasks for: {role}")
    try:
        role_agent = agents[role]
        tasks = generate_tasks_and_estimates(role, role_agent)
        tasks_and_estimates[role] = tasks
        
        # Print results immediately so we can see progress
        print(f"\n=== {role.replace('_', ' ').title()} Tasks and Estimates ===")
        print(tasks)
    except Exception as e:
        print(f"Error processing role {role}: {e}")

Creating new agents dictionary...
Created agents for roles: ['project_manager', 'requirements_engineer', 'system_engineer', 'software_engineer', 'test_engineer', 'documentation_engineer']

Generating tasks for: project_manager

=== Project Manager Tasks and Estimates ===
**Chicago WideCast Smart-Home Services Project Plan**

| **Task ID** | **Task Description** | **Amount of Work** | **Productivity Rate** |
| --- | --- | --- | --- |
| 1.1 | Review and document existing business process workflows, including roles and tasks | 40 hours | 2 pages/hour |
| 1.2 | Develop a comprehensive project plan, including stakeholder engagement, resource allocation, and risk management | 30 hours | 1.5 pages/hour |
| 1.3 | Conduct stakeholder analysis and identify key stakeholders for the project | 10 hours | 0.5 pages/hour |
| 1.4 | Develop a change management plan to ensure seamless integration of Conversational AI Assistant with existing systems | 25 hours | 1 page/hour |
| 1.5 | Collaborate with dev

In [ ]:
# Analysis of results for Experiment 1
print("Analysis of Experiment 1 Results:")
print("--------------------------------")
print(f"Platform: Ollama [Self-Hosted]")
print(f"Model: llama3.2:3b [2.0 GB]")
print(f"Embedding: HuggingFaceEmbedding BAAI/bge-small-en-v1.5")
print(f"Framework: LangChain/LangGraph")
print("\nStrengths:")
print("1. Successfully generated detailed task lists and time estimates for all roles")
print("2. Project Manager tasks were well-structured with specific hour estimates and productivity rates")
print("3. Good breakdown of tasks by role with appropriate specialized focus areas")
print("4. Maintained consistent formatting for task tables across different roles")
print("5. Included realistic assumptions and productivity rates for most roles")

print("\nWeaknesses:")
print("1. The tagged requirements were not properly formatted with REQ-XXX identifiers as requested")
print("2. The agent initially encountered multiple errors during execution requiring fixes")
print("3. Some productivity rates were inconsistent across roles (e.g., pages/hour varied significantly)")
print("4. The Requirements Engineer output included placeholder dates (2023) rather than current dates")
print("5. The attempt to generate numbered requirements used an agent approach that produced generic requirements rather than specific ones from the document")

print("\nQuality of Tagged Requirements:")
print("The quality of tagged requirements was poor. Instead of generating properly tagged requirements with REQ-XXX and UC-XXX identifiers as requested, the model produced a paragraph summarizing capabilities. While it attempted to use the Requirements_Database action to query for requirements and use cases, it failed to properly format them with unique tags/numbers according to the assignment instructions. The requirements also appeared to be generated rather than extracted from the actual WideCast document, focusing on generic smart home capabilities like energy monitoring that weren't specifically mentioned in the original requirements.")

print("\nQuality of Task Estimates:")
print("The task estimates were of moderate to good quality. The model successfully generated structured tables with task descriptions, estimated work hours, and productivity rates for all six roles. The Project Manager tasks were particularly well-defined with 15 specific tasks totaling 320 hours. Other roles also received reasonable estimates: Requirements Engineer (210 hours), System Engineer (440 hours), Software Engineer (360 hours), Test Engineer (unspecified total but detailed task breakdown), and Documentation Engineer (150 hours). The productivity rates were generally reasonable, though they varied significantly across roles without clear justification. Overall, the task estimation portion was successful, but could be improved with more consistent methodology across roles.")

Analysis of Experiment 1 Results:
--------------------------------
Platform: Ollama [Self-Hosted]
Model: llama3.2:3b [2.0 GB]
Embedding: HuggingFaceEmbedding BAAI/bge-small-en-v1.5
Framework: LangChain/LangGraph

Strengths:
1. Successfully generated detailed task lists and time estimates for all roles
2. Project Manager tasks were well-structured with specific hour estimates and productivity rates
3. Good breakdown of tasks by role with appropriate specialized focus areas
4. Maintained consistent formatting for task tables across different roles
5. Included realistic assumptions and productivity rates for most roles

Weaknesses:
1. The tagged requirements were not properly formatted with REQ-XXX identifiers as requested
2. The agent initially encountered multiple errors during execution requiring fixes
3. Some productivity rates were inconsistent across roles (e.g., pages/hour varied significantly)
4. The Requirements Engineer output included placeholder dates (2023) rather than curren

: 